In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
from pyspark.ml.feature import Normalizer, StandardScaler
import random
import pandas as pd
import time
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
from tqdm import tqdm
import warnings
import pandas as pd
import json


start_time = time.time()

kafka_topic_name = "songTopic007"
kafka_bootstrap_servers = 'localhost:9092'

spark = SparkSession \
        .builder \
        .appName("Spotify Streaming Reccomendation System") \
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
print(1)
# Construct a streaming DataFrame that reads from test-topic



if __name__ == '__main__':   
    songs_df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
        .option("subscribe", kafka_topic_name) \
        .option("startingOffsets", "earliest") \
        .load()
    
    songs_df1 = songs_df.selectExpr("CAST(value AS STRING)", "timestamp")

    songs_schema_string = "order_id INT,id STRING, name STRING,popularity INT, duration_ms DOUBLE, explicit INT, " \
                           + "artists STRING, id_artists STRING, release_date STRING, " \
                           + "danceability DOUBLE," \
                           + "energy DOUBLE, key INT, loudness DOUBLE, " \
                           + "mode INT," \
                           + "speechiness DOUBLE," \
                           + "acousticness DOUBLE, instrumentalness DOUBLE, liveness DOUBLE, " \
                           + "valence DOUBLE, tempo DOUBLE, time_signature DOUBLE"


    
    songs_df2 = songs_df1.select(from_json(col("value"), songs_schema_string).alias("song"), "timestamp")
    

    songs_df3 = songs_df2.select("song.*", "timestamp")
    songs_df3.createOrReplaceTempView("song_find");
    song_find_text = spark.sql("SELECT * FROM song_find")
    songs_agg_write_stream = song_find_text \
            .writeStream \
            .trigger(processingTime='5 seconds') \
            .outputMode("append") \
            .option("truncate", "false") \
            .format("memory") \
            .queryName("testedTable") \
            .start()

    songs_agg_write_stream.awaitTermination(1)
    print("final")
print("--- %s seconds ---" % (time.time() - start_time))




1
final
--- 12.009117126464844 seconds ---


[Stage 0:>                                                          (0 + 1) / 1]

In [4]:
from spotify_api import getSong
song_data=None
song_data = getSong.passs()
song_data.rename(columns={'duration_s': 'duration_ms' }, inplace=True)
song_data = song_data.drop(['id', 'added_at', 'time_signature','duration_ms'], axis='columns')
song_data
#rand_n = random.randint(0,len(song_data)-1)
#add_df = song_data.head(rand_n)[-1:]
add_df_temp=song_data.head(1)[-1:]
add_df= add_df_temp[['name','popularity','artists','danceability','energy',\
                'key','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]
add_df

33 tracks


,name,popularity,artists,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Ghost,90,Justin Bieber,0.601,0.741,2,-5.569,0.0478,0.185,0.000029,0.415,0.441,153.96


In [5]:
start_time = time.time()

df = spark.sql("SELECT * FROM testedTable")

df = df.sort(df.release_date.desc())


df_stream = df

df = df.drop('order_id',
 'id',
 'explicit',
  'mode',
 'release_date',
 'id_artists',
 'time_signature',
 'duration_ms',
 'timestamp')

df_sp = spark.createDataFrame(add_df)

df = df.union(df_sp)

from pyspark.ml.feature import VectorAssembler
assembler=VectorAssembler(inputCols=[
 'danceability',
 'energy',
 'loudness',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo'], outputCol='features')
assembled_data=assembler.setHandleInvalid("skip").transform(df)

from pyspark.ml.feature import StandardScaler
scale=StandardScaler(inputCol='features',outputCol='standardized')
data_scale=scale.fit(assembled_data)
df=data_scale.transform(assembled_data)

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')


KMeans_algo=KMeans(featuresCol='standardized', k=3)
    
KMeans_fit=KMeans_algo.fit(df)
    
output_df =KMeans_fit.transform(df)
  

warnings.filterwarnings("ignore")


    
    
datad = output_df.select('name',
 'artists',
 'danceability',
 'energy',
 'key',
 'loudness',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'prediction')




datf = datad.toPandas()
class SpotifyRecommender():
    def __init__(self, rec_data):
        self.rec_data_ = rec_data
    
    def spotify_recommendations(self, song_name, amount=10):
        distances = []
        
        song = self.rec_data_[(self.rec_data_.name.str.lower() == song_name.lower())].head(1).values[0]
       
        res_data = self.rec_data_[self.rec_data_.name.str.lower() != song_name.lower()]
        for r_song in tqdm(res_data.values):
           
            dist = 0
            for col in np.arange(len(res_data.columns)):
                
                if not col in [0,1,13]:
                   
                    dist = dist + np.absolute(float(song[col]) - float(r_song[col]))
                   
            distances.append(dist)
        res_data['distance'] = distances
        res_data = res_data.sort_values('distance')
        
        columns = ['name', 'artists', 'acousticness', 'liveness', 'instrumentalness', 'energy', 'danceability', 'valence']
        
        return res_data[columns][0:amount]
recommender = SpotifyRecommender(datf)
x = add_df['name'].tolist()[0]
rec_song = recommender.spotify_recommendations('titanium', 10)

v = add_df[['name', 'artists',  'acousticness', 'liveness', 'instrumentalness', 'energy', 
       'danceability', 'valence']]

rec_song = pd.concat([rec_song, v])
rec_song.to_csv('rec_song.csv')
rec_song
print("--- %s seconds ---" % (time.time() - start_time))
rec_song

100%|███████████████████████████████████████| 69601/69601 [00:02<00:00, 33540.53it/s]


--- 30.092845916748047 seconds ---


,name,artists,acousticness,liveness,instrumentalness,energy,danceability,valence
53610,"I Won't Say (I'm In Love) - From ""Hercules"" / ...",SusanEganCherylFreemanLaChanzeVaneeseThomasLil...,0.2080,0.1400,0.000000,0.471,0.684,0.453
65117,Lady (You Bring Me Up),Commodores,0.2000,0.0996,0.000339,0.593,0.892,0.867
66503,Ella Fue (She Was The One),FaniaAllStars,0.3800,0.0442,0.025800,0.648,0.824,0.971
9103,Sağı Solu Kes,Gazapizm,0.0592,0.1260,0.000608,0.438,0.937,0.457
65407,Misunderstanding,Genesis,0.2340,0.2000,0.000000,0.828,0.645,0.774
21376,Gayé,FouKi,0.0656,0.0734,0.000002,0.369,0.966,0.849
47467,I'd Rather,LutherVandross,0.2360,0.1130,0.000000,0.467,0.705,0.522
21180,Fever,OscarandtheWolf,0.0250,0.1060,0.000114,0.668,0.667,0.475
7374,Friendly Fire - Prod. Avepack,BoulevardDepo,0.3260,0.1400,0.000002,0.508,0.787,0.338
19978,Bass,RIN,0.1160,0.4540,0.000000,0.480,0.926,0.572
